## 1st Class: Cross validation and initial randomness

> Date: June 30, 2020

- **Holdout:** Technique where we separate train and test data.
- **Cross validation:** Technique used to enrich the data usage, seizing all data in both, train and test.

### Fourth project of the first class ([1_introduction_to_the_classification_with_SKLearn](https://github.com/BrunaMS/machine_learning_course/blob/master/1_introduction_to_the_classification_with_SKLearn/5.%20The%20fourth%20project.ipynb))

In this project, I made a project to verify if, with a defined price, the car model and its age, one person would sell his/her automobile, making a prediction about the sale and if is or not likely that it'll be sold.

In [1]:
import pandas as pd
from datetime import datetime
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
data = pd.read_csv(uri)

sold_change = {
    'yes' : 1,
    'no'  : 0
}

current_year = datetime.today().year
data['model_age'] = current_year - data.model_year 
data['km_per_year'] = data.mileage_per_year * 1.60934 
data.sold = data.sold.map(sold_change)
data = data.drop(columns = ['Unnamed: 0', 'mileage_per_year', 'model_year'], axis=1) # Axis: 1 - Column, 0 - Row
data.head()

,price,sold,model_age,km_per_year
0,30941.02,1,20,35085.22134
1,40557.96,1,22,12622.05362
2,89627.50,0,14,11440.79806
3,95276.14,0,5,43167.32682
4,117384.68,1,6,12770.11290


In [2]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import  StandardScaler
from sklearn.dummy import DummyClassifier
import numpy as np

x = data[['model_age', 'price', 'km_per_year']]
y = data['sold']

SEED = 15
np.random.seed(SEED)
raw_train_x, raw_test_x, train_y, test_y = train_test_split(x, y, random_state = SEED, test_size = 0.25, stratify = y)

# Baseline (default - stratified)
dummy = DummyClassifier()
dummy.fit(raw_train_x, train_y)
predictions = dummy.predict(raw_test_x)
accuracy = dummy.score(test_y, predictions)
print("The accuracy of Dummy algorithm is %.2f%% " % (accuracy*100))

The accuracy of Dummy algorithm is 51.68% 


In [3]:
scaler = StandardScaler()
scaler.fit(raw_train_x)
train_x = scaler.transform(raw_train_x)
test_x = scaler.transform(raw_test_x)

# Using SVC
model = SVC()
model.fit(train_x, train_y)
predictions = model.predict(test_x)

accuracy = accuracy_score(test_y, predictions)
print("The accuracy of this algorithm is %.2f%% " % (accuracy*100))

The accuracy of this algorithm is 77.20% 


In [4]:
## Decision trees
## Classifiers that can show to us reasons to the taken decisions.

from sklearn.tree import DecisionTreeClassifier

raw_train_x, raw_test_x, train_y, test_y = train_test_split(x, y, random_state = SEED, test_size = 0.25, stratify = y)

# Using a decision tree machine  
# The argument max_depth is used to define de max size of the thee
tree_model = DecisionTreeClassifier(max_depth = 3)
tree_model.fit(raw_train_x, train_y)
predictions = tree_model.predict(raw_test_x)

accuracy = accuracy_score(test_y, predictions)
print("The accuracy of this algorithm is %.2f%% " % (accuracy*100))

The accuracy of this algorithm is 79.68% 


### Continuation of the project with cross validation

> There are same pappers saying that a cross validation between 5 and 10  is the best way to use this tool.

In [5]:
# Import cross_validate
# Define the model
# Use cross validate with cv = 3
# Print results
# Remove train score of results
# Print test_score
# Remember to define always the same SEED
# Print mean with +/- standard deviation (std)

In [6]:
import pandas as pd
from datetime import datetime


uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
data = pd.read_csv(uri)

sold_change = {
    'yes' : 1,
    'no'  : 0
}

current_year = datetime.today().year
data['model_age'] = current_year - data.model_year 
data['km_per_year'] = data.mileage_per_year * 1.60934 
data.sold = data.sold.map(sold_change)
data = data.drop(columns = ['Unnamed: 0', 'mileage_per_year'], axis=1) # Axis: 1 - Column, 0 - Row

In [7]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

SEED = 25
np.random.seed(SEED)

x = data[['model_age', 'price', 'km_per_year']]
y = data['sold']


tree_model = DecisionTreeClassifier(max_depth = 2)
result = cross_validate(tree_model, x, y, cv = 5)
result

{'fit_time': array([0.01257825, 0.00886154, 0.00890112, 0.00751424, 0.00665474]),
 'score_time': array([0.00297451, 0.00137329, 0.00144005, 0.00257587, 0.00135517]),
 'test_score': array([0.756 , 0.7565, 0.7625, 0.7545, 0.7595])}

In [8]:
print('Test Score: ', result['test_score'])
accuracy_mean = result['test_score'].mean()
accuracy_std = result['test_score'].std()
print('Test accuracy gap: between %.2f - %.2f' % (100 * (accuracy_mean - accuracy_std), 100 * (accuracy_mean + accuracy_std)))

Test Score:  [0.756  0.7565 0.7625 0.7545 0.7595]
Test accuracy gap: between 75.49 - 76.07


> At this moment, the seed is not influencing the test score, only times are changing when run the code again

## 2nd Class: Kfold with randomization

> Date: July 01, 2020

Now, we can see that your data is been better used, sharing data between train and test and using all results to do an mean of the results, what give us more accurated and reliable outcome.

However, if we considerate that the data can be sorted by an caracteristic (like price, size, year etc.), maybe the result would be better if we mix before, ensuring that the result won't be influenced by anything.

### Splitter Classes
- **[KFold:](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)** Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default). Each fold is then used once as a validation while the k - 1 remaining folds form the training set.

In [9]:
# Create a cv with KFold (sklearn.model_selection)
# Create a function to print results
    # Print result, average and interval with +/- standard validation
# Set shuffle = True

In [10]:
def print_results(result):
    print('Raw result: ', result)
    print('\n----------------------------------------------------------------------------\nTest Score mean: %.2f%% ' % (result['test_score'].mean() * 100))
    accuracy_mean = result['test_score'].mean()
    accuracy_std = result['test_score'].std()
    print('Test accuracy gap: [%.2f%% - %.2f%%]' % (100 * (accuracy_mean - accuracy_std), 100 * (accuracy_mean + accuracy_std)))

In [11]:
from sklearn.model_selection import KFold
SEED = 20
cross_val = KFold(n_splits = 10, shuffle = True, random_state = SEED)
result = cross_validate(tree_model, x, y, cv = cross_val)
print_results(result)

Raw result:  {'fit_time': array([0.01272964, 0.01140523, 0.00732541, 0.00848174, 0.00723028,
       0.01016068, 0.00797486, 0.00737405, 0.00907803, 0.00717688]), 'score_time': array([0.00192785, 0.00246143, 0.00124002, 0.00130987, 0.00119352,
       0.00124478, 0.00138068, 0.00116658, 0.0014112 , 0.00154829]), 'test_score': array([0.738, 0.753, 0.766, 0.752, 0.75 , 0.783, 0.77 , 0.757, 0.748,
       0.761])}

----------------------------------------------------------------------------
Test Score mean: 75.78% 
Test accuracy gap: [74.57% - 76.99%]


## 3rd Class: Stratification

> Date: July 01, 2020

Considering the same situation cited above, we can have a problem with something like all the data of a given state is into the train split and all the data with another characteristic in test, for example.

Therefore, we'll run a simulation of this situation to verify what happens with our classifier in this case. 

**[Stratified KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold):** A splitter class that balance/stratify the data to train and test.

In [12]:
sorted_data = data.sort_values('sold')
sorted_data.head()

,model_year,price,sold,model_age,km_per_year
4999,2006,74023.29,0,14,24812.80412
5322,2005,84843.49,0,15,23095.63834
5319,1999,83100.27,0,21,36240.72746
5316,2002,87932.13,0,18,32249.56426
5315,2003,77937.01,0,17,28414.50704


In [13]:
x = sorted_data[['model_age', 'price', 'km_per_year']]
y = sorted_data['sold']

### 1. With KFold, without shuffle

In [14]:
cross_val = KFold(n_splits = 10, shuffle = False, random_state = SEED)
result = cross_validate(tree_model, x, y, cv = cross_val)
print_results(result)

Raw result:  {'fit_time': array([0.01356483, 0.0070827 , 0.00846028, 0.00744915, 0.00883794,
       0.00843406, 0.00711441, 0.0089643 , 0.00737357, 0.00707555]), 'score_time': array([0.00158954, 0.00141311, 0.00155973, 0.00190592, 0.00130105,
       0.00137639, 0.00128675, 0.00166392, 0.00261331, 0.00123048]), 'test_score': array([0.447, 0.409, 0.438, 0.446, 0.694, 0.663, 0.668, 0.673, 0.67 ,
       0.676])}

----------------------------------------------------------------------------
Test Score mean: 57.84% 
Test accuracy gap: [46.07% - 69.61%]


### 2. With KFold and shuffle

In [15]:
cross_val = KFold(n_splits = 10, shuffle = True, random_state = SEED)
result = cross_validate(tree_model, x, y, cv = cross_val)
print_results(result)

Raw result:  {'fit_time': array([0.00915146, 0.01149654, 0.00948882, 0.00792027, 0.00753403,
       0.00803781, 0.00708985, 0.00875521, 0.00770593, 0.01051641]), 'score_time': array([0.00422263, 0.00169539, 0.00112438, 0.00137234, 0.00121784,
       0.00133991, 0.00120234, 0.00124955, 0.00219631, 0.00112295]), 'test_score': array([0.749, 0.785, 0.749, 0.761, 0.757, 0.763, 0.758, 0.743, 0.761,
       0.752])}

----------------------------------------------------------------------------
Test Score mean: 75.78% 
Test accuracy gap: [74.69% - 76.87%]


### 3. With Stratified KFold and without shuffle

In [16]:
from sklearn.model_selection import StratifiedKFold

cross_val = StratifiedKFold(n_splits = 10, shuffle = False, random_state = SEED)
result = cross_validate(tree_model, x, y, cv = cross_val)
print_results(result)

Raw result:  {'fit_time': array([0.01151586, 0.00899553, 0.00765443, 0.01240277, 0.01059175,
       0.00798893, 0.00747681, 0.00837541, 0.00732684, 0.0089488 ]), 'score_time': array([0.00137401, 0.00151801, 0.00199819, 0.00136805, 0.0013535 ,
       0.00319219, 0.00129104, 0.00129771, 0.00153136, 0.00115895]), 'test_score': array([0.744, 0.759, 0.763, 0.765, 0.754, 0.742, 0.771, 0.748, 0.764,
       0.768])}

----------------------------------------------------------------------------
Test Score mean: 75.78% 
Test accuracy gap: [74.81% - 76.75%]


### 4. With Stratified KFold and shuffle

In [17]:
cross_val = StratifiedKFold(n_splits = 10, shuffle = True, random_state = SEED)
result = cross_validate(tree_model, x, y, cv = cross_val)
print_results(result)

Raw result:  {'fit_time': array([0.01361513, 0.00874066, 0.0103271 , 0.00858355, 0.00793099,
       0.0073328 , 0.00811601, 0.00718999, 0.00902438, 0.00750923]), 'score_time': array([0.00278401, 0.00342345, 0.00167155, 0.00112176, 0.00130582,
       0.00121593, 0.0012939 , 0.00115418, 0.00155902, 0.00197148]), 'test_score': array([0.759, 0.731, 0.761, 0.76 , 0.757, 0.764, 0.767, 0.75 , 0.753,
       0.776])}

----------------------------------------------------------------------------
Test Score mean: 75.78% 
Test accuracy gap: [74.65% - 76.91%]


### With these tests, we can conclude that:

- The shuffle is very important when there isn't any way to mix the data

- The KFold with shuffle makes a good job in this case and, if the coder prefer, appears that it wouldn't make much difference in the results

- The stratification is very good to balance/equilibrate data, mainly when we have so much data of one case and least of other.

In [18]:
sold_1 = len(data.query('sold == 1'))
sold_0 = len(data.query('sold == 0'))

print('Quantity of sold cars: %d' % sold_1)
print('Quantity of not sold cars: %d' % sold_0)

Quantity of sold cars: 5800
Quantity of not sold cars: 4200


## 4th Class: Groupable data

> Date: July 02, 2020

Now, thinking about the solution of this project, we have to consent that, if we want a useful algorithm, we have to be able to add new cars or users or anything to our dataset, right? moreover, we have to be able to apply your model in new cases, because this is our main target, making a code really useful in the real world. 

In [39]:
# Create a random column into a range between [-2, 3] and sum that with the model age
# Place this column into the dataframe 
# Verify if there is any value less than 0 and modify to be greater than 0

In [40]:
np.random.seed(SEED)
random_model = np.random.randint(-2, 3, size = len(data))
data['random_model'] = random_model + data['model_age']
data.head()

,model_year,price,sold,model_age,km_per_year,random_model
0,2000,30941.02,1,20,35085.22134,21
1,1998,40557.96,1,22,12622.05362,22
2,2006,89627.50,0,14,11440.79806,16
3,2015,95276.14,0,5,43167.32682,5
4,2014,117384.68,1,6,12770.11290,5


In [52]:
print(np.sort(data['random_model'].unique()))
print(data['random_model'].value_counts())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
20    895
19    817
18    771
21    706
16    692
17    691
15    659
14    588
22    559
13    554
12    470
11    414
10    379
23    370
9     339
8     280
7     205
24    182
6     164
5     112
4      81
3      49
2      16
1       7
Name: random_model, dtype: int64


- Now, we have to make a model that can agroup the data of tran and test according to the model. Going to the sklearn documentation, we can see the [GroupKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html), that is completely able to do this task.

In [55]:
from sklearn.model_selection import GroupKFold

cross_val = GroupKFold(n_splits = 10)
result = cross_validate(tree_model, x, y, cv = cross_val, groups = data.random_model)
print_results(result)

Raw result:  {'fit_time': array([0.00990844, 0.01103616, 0.00945783, 0.00997758, 0.00875545,
       0.00674415, 0.00709558, 0.00687957, 0.00694609, 0.00699949]), 'score_time': array([0.00158811, 0.00230002, 0.0042026 , 0.00132871, 0.0013175 ,
       0.00132084, 0.00108743, 0.00131655, 0.00114965, 0.00119781]), 'test_score': array([0.76961271, 0.76860347, 0.76646707, 0.74596774, 0.7671093 ,
       0.75145631, 0.75218659, 0.73041709, 0.75231244, 0.7734375 ])}

----------------------------------------------------------------------------
Test Score mean: 75.78% 
Test accuracy gap: [74.49% - 77.06%]
